In [1]:
import json, glob
import osgeo.ogr as ogr
import numpy as np
import rasterio as rio
import rasterio.features as features
import rasterio.warp as warp
from shapely.geometry import Polygon
from shapely.wkb import loads as wkb_loads
from shapely.ops import unary_union, polygonize


In [2]:
imagens = glob.glob('*.tif')
imagens

['mapbiomas-matogrossodosul-2018-corte.tif']

In [22]:
def open_raster(imagens):
    crs_affine_all = []
    img_crs_all = []
    geom_all = {}
    
    for img in imagens:
        
        img_open = rio.open(img)
        img_arr = img_open.read() # array das img
        crs_affine_all.append(img_open.transform) # pega as coordenadas das img
        img_crs_all.append(img_open.crs) #pega os crs das img
        
        name = img.split(".")[0]
        geom_all[name] = []
        names = [name]
        
        for geom, val in features.shapes(img_arr, transform = img_open.transform):

            geom = warp.transform_geom(img_open.crs, 'EPSG:4326', geom, precision=5)
            geom_all[name].append(geom)
    
    return img, crs_affine_all, img_crs_all, geom_all, names

img, crs_affine_all, img_crs, geom_all, names = open_raster(imagens)

In [25]:
geom_wkb_all = []
geom_dict = {}

for key,pol in geom_all.items():

   
    for item in pol:

        json_str = json.dumps(item)
        ogr_geom = ogr.CreateGeometryFromJson(json_str)
        geom_wkb = wkb_loads(ogr_geom.ExportToWkb())
        geom_wkb_all.append(geom_wkb)
    geom_dict[key] = geom_wkb_all
    


In [13]:
def rasterio_to_shapely(geom_all):
    geom_ogr_all = []
    
    
    for key,pol in geom_all.items():
        
        for item in pol:
            
            json_str = json.dumps(item)
            ogr_geom = ogr.CreateGeometryFromJson(json_str)
            geom_ogr = wkb_loads(ogr_geom.ExportToWkb())
            geom_ogr_all.append(geom_ogr)
            
    return geom_ogr, geom_ogr_all, ogr_geom, json_str

In [15]:
crs_affine_all, img_crs, geom_all, names = open_raster(imagens)
geom_ogr, geom_ogr_all, geom_ogr, json_str = rasterio_to_shapely(geom_all)

In [ ]:
def export_pol_to_shapefile(unions):
    
    driver_pol = ogr.GetDriverByName("ESRI Shapefile")
    data_source_pol = driver_pol.CreateDataSource("mapbiomas_polygon.shp")
    
#     source_pol = osr.SpatialReference()
#     source_pol.ImportFromEPSG(32722)

    target_pol = osr.SpatialReference()
    target_pol.ImportFromEPSG(4326)
    
#     ogr_transform_pol = osr.CoordinateTransformation(source_pol, target_pol)


    layer = data_source_pol.CreateLayer("geom_ogr_all", target_pol, ogr.wkbMultiPolygon)


    field_id = ogr.FieldDefn("id", ogr.OFTInteger)
    layer.CreateField(field_id_estr)
    
    field_bloco_estr = ogr.FieldDefn("imagem_name",ogr.OFTString)
    field_bloco_estr.SetWidth(20)
    layer.CreateField(field_bloco_estr)

    field_area_estr = ogr.FieldDefn("area", ogr.OFTReal)
    layer.CreateField(field_area_estr)
    
    for count, (union_image, nome_bloco) in enumerate():
        

        geoms_estradas_wkb = union_image.wkb
        ogr_geoms_estradas = ogr.CreateGeometryFromWkb(geoms_estradas_wkb)
        ogr_geoms_estradas.Transform(ogr_transform_pol)

        area = ogr_geoms_estradas.Area()

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("id_estr", count+1)
        feature.SetField("bloco", nome_bloco)
        feature.SetField("area", area)
        feature.SetGeometry(ogr_geoms_estradas)
        layer.CreateFeature(feature)
    
    
    return layer.SyncToDisk()
